In [1]:
from examples.ImageCaption import *
from pyspark.sql import SQLContext
from pyspark import SparkConf,SparkContext
from itertools import izip_longest
from com.yahoo.ml.caffe.Conversions import *
from com.yahoo.ml.caffe.Vocab import *
from com.yahoo.ml.caffe.Config import *
import json

## Create the input dataframe from cocodataset

In [3]:
df_embedded_input = sqlContext.read.parquet("/tmp/coco/parquet/df_embedded_train2014/")

In [8]:
df_images = df_embedded_input.select("data.image","data.height", "data.width", "id")
model="/tmp/coco/parquet/lrcn_coco.model"
imagenet="/Users/mridul/bigml/CaffeOnSpark/data/deploy.prototxt"
lstmnet="/Users/mridul/bigml/CaffeOnSpark/data/lrcn_word_to_preds.deploy.prototxt"
vocab="/Users/mridul/bigml/CaffeOnSpark/data/vocab.txt"

In [9]:
df_results = get_predictions(sqlContext,df_images,model,imagenet,lstmnet,vocab)
df_results.show()

+------+--------------------+
|    id|          prediction|
+------+--------------------+
| 97524|A display is , in...|
| 99876|A display is , in...|
|107748|A display is , in...|
|112497|A display is , in...|
|116799|A display is , in...|
|670231|A front bear is w...|
|674053|A front bear is w...|
|677107|A front bear is w...|
|679756|A front bear is w...|
|680179|A front bear is w...|
|564569|A plastic is pole...|
|565634|A plastic is pole...|
|566438|A plastic is pole...|
|568937|A plastic is pole...|
|569483|A plastic is pole...|
|217034|A room is door so...|
|220994|A room is door so...|
|228659|A room is door so...|
|230015|A room is door so...|
|232226|A room is door so...|
+------+--------------------+
only showing top 20 rows



In [13]:
df=df_embedded_input.join(df_results, df_embedded_input.id == df_results.id)

In [ ]:
from com.yahoo.ml.caffe.DisplayUtils import *
show_captions(df)